In [1]:
# Read the dataset description
import gzip
# Read or generate p2h, a dictionary of image name to image id (picture to hash)
import pickle
import platform
import random
# Suppress annoying stderr output when importing keras.
import sys
from lapjv import lapjv
from math import sqrt
# Determine the size of each image
from os.path import isfile

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as pil_image
from imagehash import phash
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten 
from keras.layers import GlobalMaxPooling2D, Lambda, MaxPooling2D, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
from pandas import read_csv
from scipy.ndimage import affine_transform
from tqdm import tqdm_notebook as tqdm
import time

Using TensorFlow backend.


In [2]:
TRAIN_DF = '../data/train.csv'
SUB_Df = '../data/sample_submission.csv'
TRAIN = '../data/train/'
TEST = '../data/test/'
P2H = '../data/p2h.pickle'
P2SIZE = '../data/p2size.pickle'
BB_DF = "../data/bounding_boxes.csv"
tagged = dict([(p, w) for _, p, w in read_csv(TRAIN_DF).to_records()])
submit = [p for _, p, _ in read_csv(SUB_Df).to_records()]
join = list(tagged.keys()) + submit

In [3]:
def expand_path(p):
    if isfile(TRAIN + p):
        return TRAIN + p
    if isfile(TEST + p):
        return TEST + p
    return p

In [4]:
if isfile(P2SIZE):
    print("P2SIZE exists.")
    with open(P2SIZE, 'rb') as f:
        p2size = pickle.load(f)
else:
    p2size = {}
    for p in tqdm(join):
        size = pil_image.open(expand_path(p)).size
        p2size[p] = size

P2SIZE exists.


In [5]:
def match(h1, h2):
    for p1 in h2ps[h1]:
        for p2 in h2ps[h2]:
            i1 = pil_image.open(expand_path(p1))
            i2 = pil_image.open(expand_path(p2))
            if i1.mode != i2.mode or i1.size != i2.size: return False
            a1 = np.array(i1)
            a1 = a1 - a1.mean()
            a1 = a1 / sqrt((a1 ** 2).mean())
            a2 = np.array(i2)
            a2 = a2 - a2.mean()
            a2 = a2 / sqrt((a2 ** 2).mean())
            a = ((a1 - a2) ** 2).mean()
            if a > 0.1: return False
    return True

In [6]:
if isfile(P2H):
    print("P2H exists.")
    with open(P2H, 'rb') as f:
        p2h = pickle.load(f)
else:
    # Compute phash for each image in the training and test set.
    p2h = {}
    for p in tqdm(join):
        img = pil_image.open(expand_path(p))
        h = phash(img)
        p2h[p] = h

    # Find all images associated with a given phash value.
    h2ps = {}
    for p, h in p2h.items():
        if h not in h2ps: h2ps[h] = []
        if p not in h2ps[h]: h2ps[h].append(p)

    # Find all distinct phash values
    hs = list(h2ps.keys())

    # If the images are close enough, associate the two phash values (this is the slow part: n^2 algorithm)
    h2h = {}
    for i, h1 in enumerate(tqdm(hs)):
        for h2 in hs[:i]:
            if h1 - h2 <= 6 and match(h1, h2):
                s1 = str(h1)
                s2 = str(h2)
                if s1 < s2: s1, s2 = s2, s1
                h2h[s1] = s2

    # Group together images with equivalent phash, and replace by string format of phash (faster and more readable)
    for p, h in p2h.items():
        h = str(h)
        if h in h2h: h = h2h[h]
        p2h[p] = h
#     with open(P2H, 'wb') as f:
#         pickle.dump(p2h, f)
# For each image id, determine the list of pictures
h2ps = {}
for p, h in p2h.items():
    if h not in h2ps: h2ps[h] = []
    if p not in h2ps[h]: h2ps[h].append(p)

P2H exists.


In [7]:
def show_whale(imgs, per_row=2):
    n = len(imgs)
    rows = (n + per_row - 1) // per_row
    cols = min(per_row, n)
    fig, axes = plt.subplots(rows, cols, figsize=(24 // per_row * cols, 24 // per_row * rows))
    for ax in axes.flatten(): ax.axis('off')
    for i, (img, ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))
        

def read_raw_image(p):
    img = pil_image.open(expand_path(p))
    return img

In [8]:
# For each images id, select the prefered image
def prefer(ps):
    if len(ps) == 1: return ps[0]
    best_p = ps[0]
    best_s = p2size[best_p]
    for i in range(1, len(ps)):
        p = ps[i]
        s = p2size[p]
        if s[0] * s[1] > best_s[0] * best_s[1]:  # Select the image with highest resolution
            best_p = p
            best_s = s
    return best_p

h2p = {}
for h, ps in h2ps.items():
    h2p[h] = prefer(ps)
len(h2p), list(h2p.items())[:5]

(33317,
 [('d26698c3271c757c', '0000e88ab.jpg'),
  ('ba8cc231ad489b77', '0001f9222.jpg'),
  ('bbcad234a52d0f0b', '00029d126.jpg'),
  ('c09ae7dc09f33a29', '00050a15a.jpg'),
  ('d02f65ba9f74a08a', '0005c1ef8.jpg')])

In [9]:
# Read the bounding box data from the bounding box kernel (see reference above)
p2bb = pd.read_csv(BB_DF).set_index("Image")

old_stderr = sys.stderr
sys.stderr = open('/dev/null' if platform.system() != 'Windows' else 'nul', 'w')

sys.stderr = old_stderr

img_shape = (384, 384, 1)  # The image shape used by the model
anisotropy = 2.15  # The horizontal compression ratio
crop_margin = 0.05  # The margin added around the bounding box to compensate for bounding box inaccuracy

In [10]:
def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    """
    Build a transformation matrix with the specified characteristics.
    """
    rotation = np.deg2rad(rotation)
    shear = np.deg2rad(shear)
    rotation_matrix = np.array(
        [[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix = np.array([[1.0 / height_zoom, 0, 0], [0, 1.0 / width_zoom, 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))

In [11]:
def read_cropped_image(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # If an image id was given, convert to filename
    if p in h2p:
        p = h2p[p]
    size_x, size_y = p2size[p]

    # Determine the region of the original image we want to capture based on the bounding box.
    row = p2bb.loc[p]
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    dx = x1 - x0
    dy = y1 - y0
    x0 -= dx * crop_margin
    x1 += dx * crop_margin + 1
    y0 -= dy * crop_margin
    y1 += dy * crop_margin + 1
    if x0 < 0:
        x0 = 0
    if x1 > size_x:
        x1 = size_x
    if y0 < 0:
        y0 = 0
    if y1 > size_y:
        y1 = size_y
    dx = x1 - x0
    dy = y1 - y0
    if dx > dy * anisotropy:
        dy = 0.5 * (dx / anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx = 0.5 * (dy * anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5 * img_shape[0]], [0, 1, -0.5 * img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0) / img_shape[0], 0, 0], [0, (x1 - x0) / img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
        trans = np.dot(build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.8, 1.0),
            random.uniform(0.8, 1.0),
            random.uniform(-0.05 * (y1 - y0), 0.05 * (y1 - y0)),
            random.uniform(-0.05 * (x1 - x0), 0.05 * (x1 - x0))
        ), trans)
    trans = np.dot(np.array([[1, 0, 0.5 * (y1 + y0)], [0, 1, 0.5 * (x1 + x0)], [0, 0, 1]]), trans)

    # Read the image, transform to black and white and comvert to numpy array
    img = read_raw_image(p).convert('L')
#     img = read_raw_image(p).convert('RGB')
    img = img_to_array(img)

    # Apply affine transformation
    matrix = trans[:2, :2]
    offset = trans[:2, 2]
#     print(img.shape[:-1], img.shape)
    img = img.reshape(img.shape[:-1])
#     img = img.reshape(-1,1)
    img = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant',
                           cval=np.average(img))
#     img = affine_transform(img, matrix, offset, output_shape=img.shape, order=1, mode='constant',
#                            cval=np.average(img))
    img = img.reshape(img_shape)

    # Normalize to zero mean and unit variance
    img -= np.mean(img, keepdims=True)
    img /= np.std(img, keepdims=True) + K.epsilon()
    return img

def read_for_training(p):
    """
    Read and preprocess an image with data augmentation (random transform).
    """
    return read_cropped_image(p, True)


def read_for_validation(p):
    """
    Read and preprocess an image without data augmentation (use for testing).
    """
    return read_cropped_image(p, False)


p = list(tagged.keys())[312]

In [12]:
def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y)  # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y)  # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y)  # no activation # Restore the number of original features
    y = Add()([x, y])  # Add the bypass connection
    y = Activation('relu')(y)
    return y


def build_model(lr, l2, activation='sigmoid'):
    ##############
    # BRANCH MODEL
    ##############
    regul = regularizers.l2(l2)
    optim = Adam(lr=lr)
    kwargs = {'padding': 'same', 'kernel_regularizer': regul}

    inp = Input(shape=img_shape)  # 384x384x1
    x = Conv2D(64, (9, 9), strides=2, activation='relu', **kwargs)(inp)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1, 1), activation='relu', **kwargs)(x)  # 48x48x128
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1, 1), activation='relu', **kwargs)(x)  # 24x24x256
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1, 1), activation='relu', **kwargs)(x)  # 12x12x384
    for _ in range(4):
        x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1, 1), activation='relu', **kwargs)(x)  # 6x6x512
    for _ in range(4):
        x = subblock(x, 128, **kwargs)

    x = GlobalMaxPooling2D()(x)  # 512
    branch_model = Model(inp, x)

    ############
    # HEAD MODEL
    ############
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')
    
   
    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    xa = branch_model(img_a)
    xb = branch_model(img_b)
    x = head_model([xa, xb])
#     x = head_model([xa])
    model = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
#     return model, branch_model, branch_model
    return model, branch_model, head_model


In [13]:
model, branch_model, head_model = build_model(64e-5, 0)

In [14]:
h2ws = {}
new_whale = 'new_whale'
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
for h, ws in h2ws.items():
    if len(ws) > 1:
        h2ws[h] = sorted(ws)

# For each whale, find the unambiguous images ids.
w2hs = {}
for h, ws in h2ws.items():
    if len(ws) == 1:  # Use only unambiguous pictures
        w = ws[0]
        if w not in w2hs: w2hs[w] = []
        if h not in w2hs[w]: w2hs[w].append(h)
for w, hs in w2hs.items():
    if len(hs) > 1:
        w2hs[w] = sorted(hs)

In [15]:
train = []  # A list of training image ids
for hs in w2hs.values():
    if len(hs) > 1:
        train += hs
random.shuffle(train)
train_set = set(train)

w2ts = {}  # Associate the image ids from train to each whale id.
for w, hs in w2hs.items():
    for h in hs:
        if h in train_set:
            if w not in w2ts:
                w2ts[w] = []
            if h not in w2ts[w]:
                w2ts[w].append(h)
for w, ts in w2ts.items():
    w2ts[w] = np.array(ts)

t2i = {}  # The position in train of each training image id
for i, t in enumerate(train):
    t2i[t] = i


In [16]:
import threading
class TrainingData(Sequence):
    def __init__(self, score, steps=1000, batch_size=32):
        """
        @param score the cost matrix for the picture matching
        @param steps the number of epoch we are planning with this score matrix
        """
        super(TrainingData, self).__init__()
        self.score = -score  # Maximizing the score is the same as minimuzing -score.
        self.steps = steps
        self.batch_size = batch_size
        for ts in w2ts.values():
            idxs = [t2i[t] for t in ts]
            for i in idxs:
                for j in idxs:
                    self.score[
                        i, j] = 10000.0  # Set a large value for matching whales -- eliminates this potential pairing
        self.on_epoch_end()

    def __getitem__(self, index):
        start = self.batch_size * index
        end = min(start + self.batch_size, len(self.match) + len(self.unmatch))
        size = end - start
        assert size > 0
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        b = np.zeros((size,) + img_shape, dtype=K.floatx())
        c = np.zeros((size, 1), dtype=K.floatx())
        j = start // 2
        for i in range(0, size, 2):
            a[i, :, :, :] = read_for_training(self.match[j][0])
            b[i, :, :, :] = read_for_training(self.match[j][1])
            c[i, 0] = 1  # This is a match
            a[i + 1, :, :, :] = read_for_training(self.unmatch[j][0])
            b[i + 1, :, :, :] = read_for_training(self.unmatch[j][1])
            c[i + 1, 0] = 0  # Different whales
            j += 1
        return [a, b], c

    def on_epoch_end(self):
        if self.steps <= 0: return  # Skip this on the last epoch.
        self.steps -= 1
        self.match = []
        self.unmatch = []
        
        num_threads = 6
        tmp   = num_threads*[None]
        threads   = []
        thread_input   = num_threads*[None]
        thread_idx = 0
        batch = self.score.shape[0] // (num_threads-1)
        for start in range(0, self.score.shape[0], batch):
            end = min(score.shape[0], start + batch)
#             print(self.score.shape)
#             print(start, end)
            thread_input[thread_idx]  = self.score[start:end, start:end]
            thread_idx += 1

        def worker(data_idx):
#             _,_,x = lapjv(thread_input[data_idx]) 
            x,_,_ = lapjv(thread_input[data_idx]) 
            tmp[data_idx] = x

#         print("Start worker threads")
        for i in range(num_threads):
            t = threading.Thread(target=worker, args=(i,), daemon=True)
            t.start()
            threads.append(t)
        for t in threads:
            if t is not None:
                t.join()
        x = np.concatenate(tmp)
#         print("LAP completed")
            
#         _, _, x = lapjv(self.score)  # Solve the linear assignment problem
#         x, _, _ = lapjv(self.score)  # Solve the linear assignment problem
        y = np.arange(len(x), dtype=np.int32)

        # Compute a derangement for matching whales
        for ts in w2ts.values():
            d = ts.copy()
            while True:
                random.shuffle(d)
                if not np.any(ts == d): break
            for ab in zip(ts, d): self.match.append(ab)

        # Construct unmatched whale pairs from the LAP solution.
        for i, j in zip(x, y):
            if i == j:
                print(self.score)
                print(x)
                print(y)
                print(i, j)
            assert i != j
#             print(i,j)
            self.unmatch.append((train[i], train[j]))

        # Force a different choice for an eventual next epoch.
        self.score[x, y] = 10000.0
        self.score[y, x] = 10000.0
        random.shuffle(self.match)
        random.shuffle(self.unmatch)
        # print(len(self.match), len(train), len(self.unmatch), len(train))
        assert len(self.match) == len(train) and len(self.unmatch) == len(train)

    def __len__(self):
        return (len(self.match) + len(self.unmatch) + self.batch_size - 1) // self.batch_size


# Test on a batch of 32 with random costs.
score = np.random.random_sample(size=(len(train), len(train)))
print(score.shape)
data = TrainingData(score)
(a, b), c = data[0]

(13623, 13623)


In [17]:
print(a,b,c)

[[[[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]

  [[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]

  [[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]

  ...

  [[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]

  [[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]

  [[ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   ...
   [ 1.4623204e-03]
   [ 1.4623204e-03]
   [ 1.4623204e-03]]]


 [[[ 5.4957807e-01]
   [ 5.5773008e-01]
   [ 5.3267264e-01]
   ...
   [ 5.0532407e-01]
   [ 4.9985623e-01]
   [ 4.8008233e-01]]

  [[ 5.8494300e-01]
   [ 5.7474321e-01]
   [ 5.5536431e-01]
   ...
   [ 4.9922189e-01]
 

In [18]:
# A Keras generator to evaluate only the BRANCH MODEL
class FeatureGen(Sequence):
    def __init__(self, data, batch_size=64, verbose=1):
        super(FeatureGen, self).__init__()
        self.data = data
        self.batch_size = batch_size
        self.verbose = verbose
        if self.verbose > 0: self.progress = tqdm(total=len(self), desc='Features')

    def __getitem__(self, index):
        start = self.batch_size * index
        size = min(len(self.data) - start, self.batch_size)
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        for i in range(size): a[i, :, :, :] = read_for_validation(self.data[start + i])
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return a

    def __len__(self):
        return (len(self.data) + self.batch_size - 1) // self.batch_size


In [19]:
class ScoreGen(Sequence):
    def __init__(self, x, y=None, batch_size=2048, verbose=1):
        super(ScoreGen, self).__init__()
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.verbose = verbose
        if y is None:
            self.y = self.x
            self.ix, self.iy = np.triu_indices(x.shape[0], 1)
        else:
            self.iy, self.ix = np.indices((y.shape[0], x.shape[0]))
            self.ix = self.ix.reshape((self.ix.size,))
            self.iy = self.iy.reshape((self.iy.size,))
        self.subbatch = (len(self.x) + self.batch_size - 1) // self.batch_size
        if self.verbose > 0:
            self.progress = tqdm(total=len(self), desc='Scores')

    def __getitem__(self, index):
        start = index * self.batch_size
        end = min(start + self.batch_size, len(self.ix))
        a = self.y[self.iy[start:end], :]
        b = self.x[self.ix[start:end], :]
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return [a, b]

    def __len__(self):
        return (len(self.ix) + self.batch_size - 1) // self.batch_size

In [20]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

def set_lr(model, lr):
    K.set_value(model.optimizer.lr, float(lr))


def get_lr(model):
    return K.get_value(model.optimizer.lr)


def score_reshape(score, x, y=None):
    """
    Tranformed the packed matrix 'score' into a square matrix.
    @param score the packed matrix
    @param x the first image feature tensor
    @param y the second image feature tensor if different from x
    @result the square matrix
    """
    if y is None:
        # When y is None, score is a packed upper triangular matrix.
        # Unpack, and transpose to form the symmetrical lower triangular matrix.
        m = np.zeros((x.shape[0], x.shape[0]), dtype=K.floatx())
        m[np.triu_indices(x.shape[0], 1)] = score.squeeze()
        m += m.transpose()
    else:
        m = np.zeros((y.shape[0], x.shape[0]), dtype=K.floatx())
        iy, ix = np.indices((y.shape[0], x.shape[0]))
        ix = ix.reshape((ix.size,))
        iy = iy.reshape((iy.size,))
        m[iy, ix] = score.squeeze()
    return m


def compute_score(verbose=1):
    """
    Compute the score matrix by scoring every pictures from the training set against every other picture O(n^2).
    """
    features = branch_model.predict_generator(FeatureGen(train, verbose=verbose), max_queue_size=12, workers=6,
                                              verbose=0)
    score = head_model.predict_generator(ScoreGen(features, verbose=verbose), max_queue_size=12, workers=6, verbose=0)
    score = score_reshape(score, features)
    return features, score


def make_steps(step, ampl):
    """
    Perform training epochs
    @param step Number of epochs to perform
    @param ampl the K, the randomized component of the score matrix.
    """
    global w2ts, t2i, steps, features, score, histories

    # shuffle the training pictures
    random.shuffle(train)

    # Map whale id to the list of associated training picture hash value
    w2ts = {}
    for w, hs in w2hs.items():
        for h in hs:
            if h in train_set:
                if w not in w2ts: w2ts[w] = []
                if h not in w2ts[w]: w2ts[w].append(h)
    for w, ts in w2ts.items(): w2ts[w] = np.array(ts)

    # Map training picture hash value to index in 'train' array    
    t2i = {}
    for i, t in enumerate(train): t2i[t] = i

    # Compute the match score for each picture pair
    features, score = compute_score()
#     np.save('../cache/features7', features)
#     np.save('../cache/score7', score)
#     features = np.load('../cache/features7.npy')
#     score = np.load('../cache/score7.npy')
    chkpt = '../cache/wt7-{epoch:02d}.h5'
    checkpoint = ModelCheckpoint(chkpt, monitor='acc', verbose=1, 
                                 save_best_only=True, mode='max', save_weights_only = True)
    reduceLROnPlat = ReduceLROnPlateau(monitor='acc', factor=0.1, patience=3, 
                                       verbose=1, mode='auto', epsilon=0.0001)
    early = EarlyStopping(monitor="acc", 
                          mode="max", 
                          patience=6)
    callbacks_list = [checkpoint, early, reduceLROnPlat]
    
    # Train the model for 'step' epochs
    history = model.fit_generator(
        TrainingData(score + ampl * np.random.random_sample(size=score.shape), steps=step, batch_size=32),
        initial_epoch=steps, epochs=steps + step, max_queue_size=12, workers=6, verbose=1,
        callbacks = callbacks_list).history
    steps += step

    # Collect history data
    history['epochs'] = steps
    history['ms'] = np.mean(score)
    history['lr'] = get_lr(model)
    print(history['epochs'], history['lr'], history['ms'])
    histories.append(history)

In [21]:
histories = []
steps = 0

if isfile('../cache/ashish.standard.model-9'):
    tmp = keras.models.load_model('../cache/ashish.standard.model-9')
    model.set_weights(tmp.get_weights())
else:
#     tmp = keras.models.load_model('../cache/ashish.standard.model-2')
#     model.set_weights(tmp.get_weights())
    # epoch -> 10
    make_steps(10, 1000)
    ampl = 100.0
    for _ in range(2):
        print('noise ampl.  = ', ampl)
        make_steps(5, ampl)
        ampl = max(1.0, 100 ** -0.1 * ampl)
    model.save('../cache/ashish.standard.model-1')
    # epoch -> 110
    for _ in range(18): make_steps(5, 1.0)
    model.save('../cache/ashish.standard.model-2')
#     # epoch -> 160
#     set_lr(model, 16e-5)
#     for _ in range(10): make_steps(5, 0.5)
#     model.save('../cache/ashish.standard.model-3')
#     # epoch -> 200
#     set_lr(model, 4e-5)
#     for _ in range(8): make_steps(5, 0.25)
#     model.save('../cache/ashish.standard.model-4')
#     # epoch -> 210
#     set_lr(model, 1e-5)
#     for _ in range(2): make_steps(5, 0.25)
#     model.save('../cache/ashish.standard.model-5')
#     # epoch -> 260
#     weights = model.get_weights()
#     model, branch_model, head_model = build_model(64e-5, 0.0002)
#     model.set_weights(weights)
#     for _ in range(10): make_steps(5, 1.0)
#     model.save('../cache/ashish.standard.model-6')
#     # epoch -> 310
#     set_lr(model, 16e-5)
#     for _ in range(10): make_steps(5, 0.5)
#     model.save('../cache/ashish.standard.model-7')
#     # epoch -> 350
#     set_lr(model, 4e-5)
#     for _ in range(8): make_steps(5, 0.25)
#     model.save('../cache/ashish.standard.model-8')
#     # epoch -> 500
#     set_lr(model, 1e-5)
#     for _ in range(30): make_steps(5, 0.25)
#     model.save('../cache/ashish.standard.model')

In [22]:
# epoch -> 600
model.load_weights('../cache/wt7-500.h5')
set_lr(model, 1e-5)
for _ in range(20): make_steps(5, 0.25)
model.save('../cache/ashish.standard.model-9')

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))

/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 501/505
852/852 [==============================] - 230s 270ms/step - loss: 0.1541 - binary_crossentropy: 0.0374 - acc: 0.9871

Epoch 00501: acc improved from -inf to 0.98712, saving model to ../cache/wt7-501.h5
Epoch 502/505
852/852 [==============================] - 239s 280ms/step - loss: 0.1414 - binary_crossentropy: 0.0248 - acc: 0.9919

Epoch 00502: acc improved from 0.98712 to 0.99189, saving model to ../cache/wt7-502.h5
Epoch 503/505
852/852 [==============================] - 239s 281ms/step - loss: 0.1358 - binary_crossentropy: 0.0194 - acc: 0.9931

Epoch 00503: acc improved from 0.99189 to 0.99306, saving model to ../cache/wt7-503.h5
Epoch 504/505
852/852 [==============================] - 241s 283ms/step - loss: 0.1321 - binary_crossentropy: 0.0158 - acc: 0.9947

Epoch 00504: acc improved from 0.99306 to 0.99471, saving model to ../cache/wt7-504.h5
Epoch 505/505
852/852 [==============================] - 240s 281ms/step - loss: 0.1298 - binary_crossentropy: 0.0138 - acc

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 591/595
852/852 [==============================] - 231s 271ms/step - loss: 0.1381 - binary_crossentropy: 0.0357 - acc: 0.9886

Epoch 00591: acc improved from -inf to 0.98862, saving model to ../cache/wt7-591.h5
Epoch 592/595
852/852 [==============================] - 240s 282ms/step - loss: 0.1274 - binary_crossentropy: 0.0252 - acc: 0.9925

Epoch 00592: acc improved from 0.98862 to 0.99248, saving model to ../cache/wt7-592.h5
Epoch 593/595
852/852 [==============================] - 241s 283ms/step - loss: 0.1228 - binary_crossentropy: 0.0206 - acc: 0.9938

Epoch 00593: acc improved from 0.99248 to 0.99376, saving model to ../cache/wt7-593.h5
Epoch 594/595
852/852 [==============================] - 241s 282ms/step - loss: 0.1175 - binary_crossentropy: 0.0154 - acc: 0.9953

Epoch 00594: acc improved from 0.99376 to 0.99530, saving model to ../cache/wt7-594.h5
Epoch 595/595
852/852 [==============================] - 241s 283ms/step - loss: 0.1167 - binary_crossentropy: 0.0149 - ac

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 596/600
852/852 [==============================] - 232s 272ms/step - loss: 0.1311 - binary_crossentropy: 0.0294 - acc: 0.9905

Epoch 00596: acc improved from -inf to 0.99046, saving model to ../cache/wt7-596.h5
Epoch 597/600
852/852 [==============================] - 238s 280ms/step - loss: 0.1233 - binary_crossentropy: 0.0217 - acc: 0.9928

Epoch 00597: acc improved from 0.99046 to 0.99281, saving model to ../cache/wt7-597.h5
Epoch 598/600
852/852 [==============================] - 239s 281ms/step - loss: 0.1175 - binary_crossentropy: 0.0160 - acc: 0.9952

Epoch 00598: acc improved from 0.99281 to 0.99519, saving model to ../cache/wt7-598.h5
Epoch 599/600
852/852 [==============================] - 240s 281ms/step - loss: 0.1161 - binary_crossentropy: 0.0148 - acc: 0.9952

Epoch 00599: acc did not improve from 0.99519
Epoch 600/600
852/852 [==============================] - 239s 280ms/step - loss: 0.1117 - binary_crossentropy: 0.0106 - acc: 0.9967

Epoch 00600: acc improved from

In [23]:
# epoch -> 750
# model.load_weights('../cache/wt7-600.h5')
set_lr(model, 1e-5)
for _ in range(30): make_steps(5, 0.25)
model.save('../cache/ashish.standard.model-9')

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))

/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 601/605
852/852 [==============================] - 231s 271ms/step - loss: 0.1369 - binary_crossentropy: 0.0359 - acc: 0.9888

Epoch 00601: acc improved from -inf to 0.98877, saving model to ../cache/wt7-601.h5
Epoch 602/605
852/852 [==============================] - 238s 280ms/step - loss: 0.1255 - binary_crossentropy: 0.0246 - acc: 0.9920

Epoch 00602: acc improved from 0.98877 to 0.99196, saving model to ../cache/wt7-602.h5
Epoch 603/605
852/852 [==============================] - 240s 281ms/step - loss: 0.1179 - binary_crossentropy: 0.0171 - acc: 0.9952

Epoch 00603: acc improved from 0.99196 to 0.99523, saving model to ../cache/wt7-603.h5
Epoch 604/605
852/852 [==============================] - 241s 283ms/step - loss: 0.1155 - binary_crossentropy: 0.0149 - acc: 0.9954

Epoch 00604: acc improved from 0.99523 to 0.99541, saving model to ../cache/wt7-604.h5
Epoch 605/605
852/852 [==============================] - 240s 282ms/step - loss: 0.1117 - binary_crossentropy: 0.0112 - acc

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 741/745
852/852 [==============================] - 229s 268ms/step - loss: 0.1118 - binary_crossentropy: 0.0270 - acc: 0.9914

Epoch 00741: acc improved from -inf to 0.99141, saving model to ../cache/wt7-741.h5
Epoch 742/745
852/852 [==============================] - 237s 278ms/step - loss: 0.1020 - binary_crossentropy: 0.0173 - acc: 0.9949

Epoch 00742: acc improved from 0.99141 to 0.99490, saving model to ../cache/wt7-742.h5
Epoch 743/745
852/852 [==============================] - 236s 277ms/step - loss: 0.0983 - binary_crossentropy: 0.0136 - acc: 0.9957

Epoch 00743: acc improved from 0.99490 to 0.99574, saving model to ../cache/wt7-743.h5
Epoch 744/745
852/852 [==============================] - 236s 277ms/step - loss: 0.0941 - binary_crossentropy: 0.0096 - acc: 0.9971

Epoch 00744: acc improved from 0.99574 to 0.99710, saving model to ../cache/wt7-744.h5
Epoch 745/745
852/852 [==============================] - 237s 278ms/step - loss: 0.0923 - binary_crossentropy: 0.0079 - ac

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 746/750
852/852 [==============================] - 228s 268ms/step - loss: 0.1137 - binary_crossentropy: 0.0294 - acc: 0.9902

Epoch 00746: acc improved from -inf to 0.99020, saving model to ../cache/wt7-746.h5
Epoch 747/750
852/852 [==============================] - 237s 278ms/step - loss: 0.1015 - binary_crossentropy: 0.0173 - acc: 0.9950

Epoch 00747: acc improved from 0.99020 to 0.99505, saving model to ../cache/wt7-747.h5
Epoch 748/750
852/852 [==============================] - 237s 278ms/step - loss: 0.0988 - binary_crossentropy: 0.0146 - acc: 0.9958

Epoch 00748: acc improved from 0.99505 to 0.99582, saving model to ../cache/wt7-748.h5
Epoch 749/750
852/852 [==============================] - 237s 278ms/step - loss: 0.0947 - binary_crossentropy: 0.0107 - acc: 0.9975

Epoch 00749: acc improved from 0.99582 to 0.99747, saving model to ../cache/wt7-749.h5
Epoch 750/750
852/852 [==============================] - 236s 277ms/step - loss: 0.0932 - binary_crossentropy: 0.0093 - ac

In [ ]:
# epoch -> 900
# model.load_weights('../cache/wt7-600.h5')
set_lr(model, 1e-5)
for _ in range(30): make_steps(5, 0.25)
model.save('../cache/ashish.standard.model-10')

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))

/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 751/755
852/852 [==============================] - 233s 273ms/step - loss: 0.1143 - binary_crossentropy: 0.0305 - acc: 0.9900

Epoch 00751: acc improved from -inf to 0.98998, saving model to ../cache/wt7-751.h5
Epoch 752/755
852/852 [==============================] - 239s 280ms/step - loss: 0.1036 - binary_crossentropy: 0.0199 - acc: 0.9936

Epoch 00752: acc improved from 0.98998 to 0.99361, saving model to ../cache/wt7-752.h5
Epoch 753/755
852/852 [==============================] - 239s 281ms/step - loss: 0.0980 - binary_crossentropy: 0.0143 - acc: 0.9958

Epoch 00753: acc improved from 0.99361 to 0.99578, saving model to ../cache/wt7-753.h5
Epoch 754/755
852/852 [==============================] - 240s 282ms/step - loss: 0.0960 - binary_crossentropy: 0.0124 - acc: 0.9963

Epoch 00754: acc improved from 0.99578 to 0.99626, saving model to ../cache/wt7-754.h5
Epoch 755/755
852/852 [==============================] - 247s 289ms/step - loss: 0.0925 - binary_crossentropy: 0.0090 - acc

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 811/815
852/852 [==============================] - 232s 272ms/step - loss: 0.1058 - binary_crossentropy: 0.0271 - acc: 0.9910

Epoch 00811: acc improved from -inf to 0.99097, saving model to ../cache/wt7-811.h5
Epoch 812/815
852/852 [==============================] - 238s 279ms/step - loss: 0.0966 - binary_crossentropy: 0.0179 - acc: 0.9941

Epoch 00812: acc improved from 0.99097 to 0.99413, saving model to ../cache/wt7-812.h5
Epoch 813/815
852/852 [==============================] - 239s 280ms/step - loss: 0.0901 - binary_crossentropy: 0.0115 - acc: 0.9963

Epoch 00813: acc improved from 0.99413 to 0.99629, saving model to ../cache/wt7-813.h5
Epoch 814/815
852/852 [==============================] - 238s 280ms/step - loss: 0.0901 - binary_crossentropy: 0.0116 - acc: 0.9966

Epoch 00814: acc improved from 0.99629 to 0.99659, saving model to ../cache/wt7-814.h5
Epoch 815/815
852/852 [==============================] - 239s 280ms/step - loss: 0.0863 - binary_crossentropy: 0.0079 - ac

HBox(children=(IntProgress(value=0, description='Features', max=213, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Scores', max=45306, style=ProgressStyle(description_width='initial')), HTML(value='')))


Epoch 816/820
852/852 [==============================] - 232s 272ms/step - loss: 0.1074 - binary_crossentropy: 0.0291 - acc: 0.9912

Epoch 00816: acc improved from -inf to 0.99119, saving model to ../cache/wt7-816.h5
Epoch 817/820
852/852 [==============================] - 240s 281ms/step - loss: 0.0991 - binary_crossentropy: 0.0208 - acc: 0.9942

Epoch 00817: acc improved from 0.99119 to 0.99420, saving model to ../cache/wt7-817.h5
Epoch 818/820
383/852 [============>.................] - ETA: 2:16 - loss: 0.0932 - binary_crossentropy: 0.0150 - acc: 0.9956

In [22]:
def prepare_submission(threshold, filename):
    """
    Generate a Kaggle submission file.
    @param threshold the score given to 'new_whale'
    @param filename the submission file name
    """
    vtop = 0
    vhigh = 0
    pos = [0, 0, 0, 0, 0, 0]
    with open(filename, 'wt', newline='\n') as f:
        f.write('Image,Id\n')
        for i, p in enumerate(tqdm(submit)):
            t = []
            s = set()
            a = score[i, :]
            for j in list(reversed(np.argsort(a))):
                h = known[j]
                if a[j] < threshold and new_whale not in s:
                    pos[len(t)] += 1
                    s.add(new_whale)
                    t.append(new_whale)
                    if len(t) == 5: break;
                for w in h2ws[h]:
                    assert w != new_whale
                    if w not in s:
                        if a[j] > 1.0:
                            vtop += 1
                        elif a[j] >= threshold:
                            vhigh += 1
                        s.add(w)
                        t.append(w)
                        if len(t) == 5: break;
                if len(t) == 5: break;
            if new_whale not in s: pos[5] += 1
            assert len(t) == 5 and len(s) == 5
            f.write(p + ',' + ' '.join(t[:5]) + '\n')
    return vtop, vhigh, pos

In [23]:
# Find elements from training sets not 'new_whale'
tic = time.time()
h2ws = {}
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
known = sorted(list(h2ws.keys()))

# Dictionary of picture indices
h2i = {}
for i, h in enumerate(known): h2i[h] = i

# Evaluate the model.
fknown = branch_model.predict_generator(FeatureGen(known), max_queue_size=20, workers=10, verbose=0)
fsubmit = branch_model.predict_generator(FeatureGen(submit), max_queue_size=20, workers=10, verbose=0)
score = head_model.predict_generator(ScoreGen(fknown, fsubmit), max_queue_size=20, workers=10, verbose=0)
score = score_reshape(score, fknown, fsubmit)



HBox(children=(IntProgress(value=0, description='Features', max=246, style=ProgressStyle(description_width='initial')), HTML(value='')))

InternalError: Dst tensor is not initialized.
	 [[Node: _arg_input_1_0_0/_12643 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_1950__arg_input_1_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: global_max_pooling2d_1/Max/_12647 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2276_global_max_pooling2d_1/Max", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
# Generate the subsmission file.
prepare_submission(0.9992, '../submissions/sub7f.csv')
toc = time.time()
print("Submission time: ", (toc - tic) / 60.)

In [ ]:
%%time
!kaggle competitions submit -c humpback-whale-identification -f ../submissions/sub7f.csv -m ""

In [ ]:
from time import sleep
# sleep(120)
!kaggle competitions submissions -c humpback-whale-identification